# Youtube Video Downloader

### Install pytube library

In [5]:
!pip install --upgrade pytube

### Run the following to avoid pytube streaming and throttling function errors:

In [19]:
from pytube.innertube import _default_clients
from pytube import cipher
import re

_default_clients["ANDROID"]["context"]["client"]["clientVersion"] = "19.08.35"
_default_clients["IOS"]["context"]["client"]["clientVersion"] = "19.08.35"
_default_clients["ANDROID_EMBED"]["context"]["client"]["clientVersion"] = "19.08.35"
_default_clients["IOS_EMBED"]["context"]["client"]["clientVersion"] = "19.08.35"
_default_clients["IOS_MUSIC"]["context"]["client"]["clientVersion"] = "6.41"
_default_clients["ANDROID_MUSIC"] = _default_clients["ANDROID_CREATOR"]



def get_throttling_function_name(js: str) -> str:
    """Extract the name of the function that computes the throttling parameter.

    :param str js:
        The contents of the base.js asset file.
    :rtype: str
    :returns:
        The name of the function used to compute the throttling parameter.
    """
    function_patterns = [
        r'a\.[a-zA-Z]\s*&&\s*\([a-z]\s*=\s*a\.get\("n"\)\)\s*&&\s*'
        r'\([a-z]\s*=\s*([a-zA-Z0-9$]+)(\[\d+\])?\([a-z]\)',
        r'\([a-z]\s*=\s*([a-zA-Z0-9$]+)(\[\d+\])\([a-z]\)',
    ]
    #logger.debug('Finding throttling function name')
    for pattern in function_patterns:
        regex = re.compile(pattern)
        function_match = regex.search(js)
        if function_match:
            #logger.debug("finished regex search, matched: %s", pattern)
            if len(function_match.groups()) == 1:
                return function_match.group(1)
            idx = function_match.group(2)
            if idx:
                idx = idx.strip("[]")
                array = re.search(
                    r'var {nfunc}\s*=\s*(\[.+?\]);'.format(
                        nfunc=re.escape(function_match.group(1))),
                    js
                )
                if array:
                    array = array.group(1).strip("[]").split(",")
                    array = [x.strip() for x in array]
                    return array[int(idx)]

    raise RegexMatchError(
        caller="get_throttling_function_name", pattern="multiple"
    )

cipher.get_throttling_function_name = get_throttling_function_name

# Run following to start download:

In [20]:
from pytube import YouTube

from google.colab import files

# Enter Links of the videos to be downloaded, seperate links by ','
links = ["https://www.youtube.com/watch?v=EwSlcff15Oc"]

for link in links:
    try:
        print(f"Processing {link}...")
        yt = YouTube(link)
        print("YouTube object created successfully.")
    except Exception as e:
        print(f"Connection Error for {link}: {e}")
        continue

    # Get all streams and filter for mp4 files
    try:
        mp4_streams = yt.streams.filter(file_extension='mp4').get_highest_resolution()
        if mp4_streams is None:
            print(f"No mp4 streams found for {link}")
            continue
        print(f"Stream fetched: {mp4_streams.resolution}")
    except Exception as e:
        print(f"Error fetching streams for {link}: {e}")
        continue

    # Get the video with the highest resolution
    d_video = mp4_streams

    try:
        # Download the video
        d_video.download('/content', filename='your_video_file.mp4')
        files.download('/content/your_video_file.mp4')
        print('Video downloaded successfully!')
    except:
        print("Some Error!")




Processing https://www.youtube.com/watch?v=EwSlcff15Oc...
YouTube object created successfully.
Stream fetched: 360p


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Video downloaded successfully!
